<a href="https://colab.research.google.com/github/ram130849/llm-finetuning/blob/main/mistral_7B_Instruct_QA_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip  install -qU openai langchain PyPDF2 Faiss-gpu transformers tiktoken pinecone-client sentence-transformers chainlit streamlit
!pip install -q rank_bm25
!pip install -qU trl Py7zr auto-gptq optimum
!pip install -qU PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.8 MB/s eta 0:00:00


In [5]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import CacheBackedEmbeddings,HuggingFaceEmbeddings
from langchain.storage import LocalFileStore
from langchain.retrievers import BM25Retriever,EnsembleRetriever
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.llms import HuggingFacePipeline
from langchain.cache import InMemoryCache
from langchain.schema import prompt
from langchain.callbacks import StdOutCallbackHandler
from langchain.vectorstores  import FAISS
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (ChatPromptTemplate,
                                    SystemMessagePromptTemplate,
                                    HumanMessagePromptTemplate)

import PyPDF2
from io import BytesIO
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [6]:
import os
from configparser import ConfigParser
env_config =  ConfigParser()

In [10]:
lamda_loader = PyPDFLoader("/content/LAMDA.pdf").load()
print(f"len of documents in :{len(lamda_loader)}")

len of documents in :47


In [11]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
#
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500,
                                      chunk_overlap=100,
                                      length_function=len,)
#
lamda_documents = text_splitter.transform_documents(lamda_loader)
print(f"number of chunks in lambda documents : {len(lamda_documents)}")

number of chunks in lambda documents : 133


In [12]:
store = LocalFileStore("./cache/")
embed_model_id = 'sentence-transformers/all-MiniLM-L6-v2'
core_embeddings_model = HuggingFaceEmbeddings(model_name=embed_model_id)
embedder = CacheBackedEmbeddings.from_bytes_store(core_embeddings_model,
                                                  store,
                                                  namespace=embed_model_id)

In [13]:
vectorstore = FAISS.from_documents(lamda_documents,embedder)

In [14]:
bm25_retriever = BM25Retriever.from_documents(lamda_documents)
bm25_retriever.k=5

In [15]:
query = "What is the parameter size for Lamda Model?"
embedding_vector = core_embeddings_model.embed_query(query)
len(embedding_vector)

384

In [16]:
docs = vectorstore.similarity_search_by_vector(embedding_vector,k=15)

In [17]:
for page in docs:
  print(page.page_content)
  print("\n")

evaluation process discussed in Solaiman et al. [87] are 0.19 for PT and 0.14 for LaMDA (lower is better).
37


64 layers, dmodel = 8192 ,dff= 65536 ,h= 128 ,dk=dv= 128 , relative attention as described in T5 [ 11], and
gated-GELU activation as described in Raffel et al. [93].
We pre-trained LaMDA on 1024 TPU-v3 chips for a total of about 57.7 days, and 256K tokens per batch. We used
the Lingvo framework [ 94] for training and achieved 123 TFLOPS/sec with 56.5% FLOPS utilization with the 2D
sharding algorithm, as described in GSPMD [ 95] (see Section 10 for carbon footprint estimates). We also trained
smaller 2B-parameter and 8B-parameter models to measure the effects of model scaling on our metrics. Hyperparameter
details for the models of different sizes can be found in Table 27, Appendix D.
Figure 2 gives an overview of the pre-training stage. We call the model before any ﬁne-tuning "PT", for PreTrained.
Figure 2: LaMDA pre-training as a language model.
4


outputs through ﬁne-tunin

In [18]:
%%timeit -n 1 -r 1
query = "What is the parameter size for Lamda Model?"
#
embedding_vector = core_embeddings_model.embed_query(query)
docs = vectorstore.similarity_search_by_vector(embedding_vector,k=15)

26.5 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [19]:
faiss_retriever = vectorstore.as_retriever(search_kwargs={"k":5})
ensemble_retriever = EnsembleRetriever(retrievers=[bm25_retriever,faiss_retriever],
                                       weights=[0.5,0.5])

In [20]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_name_or_path = "TheBloke/Mistral-7B-Instruct-v0.1-GPTQ"
# To use a different branch, change revision
# For example: revision="gptq-4bit-32g-actorder_True"
model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map="auto",
                                             trust_remote_code=False)

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

In [44]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.1,
    top_p=0.95,
    top_k=40,
    repetition_penalty=1.1
)

In [45]:
import langchain
llm = HuggingFacePipeline(pipeline=pipe)
langchain.llm_cache = InMemoryCache()

In [46]:
handler = StdOutCallbackHandler()

qa_with_sources_chain = RetrievalQAWithSourcesChain.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever = ensemble_retriever,
    callbacks=[handler]
)

In [49]:
%%time
query = "On which dataset Lamda was pretrained?"
response = qa_with_sources_chain({"question":query})
print('response:',response)
print(f"Response generated : \n {response['answer']}")
print(f"Source Documents : \n {response['sources']}")



> Entering new RetrievalQAWithSourcesChain chain...

> Finished chain.
response: {'question': 'On which dataset Lamda was pretrained?', 'answer': ' LaMDA was pre-trained on a dataset created from public dialog data and other public web documents.\n', 'sources': '/content/LAMDA.pdf'}
Response generated : 
  LaMDA was pre-trained on a dataset created from public dialog data and other public web documents.

Source Documents : 
 /content/LAMDA.pdf
CPU times: user 36.8 s, sys: 1.49 s, total: 38.3 s
Wall time: 39.6 s


In [52]:
from auto_gptq import exllama_set_max_input_length
model = exllama_set_max_input_length(model, 6024)

In [53]:
%%time
query = "On what metrics Lamda was evaluated?"
response = qa_with_sources_chain({"question":query})
print(f"Response generated : \n {response['answer']}")
print(f"Source Documents : \n {response['sources']}")
print(f"Number of Documents returned : {len(response['sources'])}")



> Entering new RetrievalQAWithSourcesChain chain...

> Finished chain.
Response generated : 
  Lamda was evaluated on several metrics including PT (2B), PT (8B), PT (137B), FT quality-safety (137B), LaMDA (2B), LaMDA (8B), and LaMDA (137B).

Source Documents : 
 38
Number of Documents returned : 2
CPU times: user 12.5 s, sys: 2.92 s, total: 15.4 s
Wall time: 17.1 s
